In [36]:
# Import your newly installed selenium package
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as soup
from selenium.webdriver.chrome.options import Options
from time import sleep
from random import randint
import datetime
import re
import numpy
import pandas as pd
import openpyxl

url = "http://www.ufc.ca/fighter/khabib-Nurmagomedov"

In [105]:
chrome_options = Options()
# chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")

In [109]:
# driver = webdriver.Chrome(chrome_options=chrome_options, executable_path="/media/dodz/Data/Linux/Development/drivers/chromedriver")
driver = webdriver.Chrome(options=chrome_options)
# A new Chrome (or other browser) window should open up

# Now just tell it wherever you want it to go
driver.get(url)


In [108]:
driver.close()

In [110]:
# Return the title
# Good to use assert clause to make sure you are where you think you are
driver.title

'Khabib Nurmagomedov - Official UFC® Fighter Profile'

In [6]:
# Save a screenshot; check and see what was going on or capture something specific
driver.save_screenshot('khabib-Nurmagomedov.png')

True

Resulted Image: 
![alt text](./khabib-Nurmagomedov.png)


In [32]:
fighters = pd.read_excel('ufc-fighters.xlsx')
fighters

,name,link,weight(kg),hieght(cm),record,nicknam
0,Shamil Abdurakhimov,/fighter/shamil-abdurakhimov,106.0,190.0,19-4-0,Abrek
1,Daichi Abe,/fighter/daichi-Abe,77.0,180.0,6-1-0,NaN
2,Juan Adams,/fighter/juan-Adams,120.0,195.0,4-0-0,The Kraken
3,Israel Adesanya,/fighter/israel-Adesanya,84.0,193.0,15-0-0,The Last Stylebender
4,Jessica Aguilar,/fighter/jessica-aguilar,52.0,162.0,20-6-0,Jag
5,Kevin Aguilar,/fighter/kevin-Aguilar,70.0,170.0,14-1-0,Angel of Death
6,Omari Akhmedov,/fighter/omari-Akhmedov,84.0,182.0,17-4-1,Wolverine
7,Junior Albini,/fighter/junior-Albini,120.0,190.0,14-4-0,Baby
8,Alexandra Albu,/fighter/alexandra-Albu,52.0,160.0,7-0-0,Stitch
9,Iuri Alcantara,/fighter/iuri-Alcantara,61.0,175.0,"35-10-0, 1NC",Marajo


In [33]:
def get_history(fighter_history):
    records = []
    event_regex = r'(?P<event>(?P<name>[^:]*):?(?P<details>.*)?(\s*)\s{2}(?P<date>(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(?P<day>\d{1,2}),\s*(?P<year>\d{4})))'
    for fight in fighter_history:
        try:
            result_element = (fight.find('td',{"class":"result"})).find('div')
            # f_type = result_element.get_attribute('class')
            result = result_element.get_text()
        except:
            result = ""
        try:
            fighters = (fight.find('td', {"class":"fighter"})).find_all('div')
            fighter1 = fighters[0].get_text().strip()
            fighter2 = fighters[1].get_text().strip()
        except:
            fighter1 = fighter2 = ""
        try:
            event = (fight.find('td', {"class":"event"})).get_text()
            event = re.sub('[\n\t]','',event)
            event_name = re.search( event_regex, event).group('name')
            event_date = re.search( event_regex, event).group('date')
            remarks = re.search( event_regex, event).group('details')
            # event = (fight.find('td', {"class":"event"})).find('div')
            # print(event_name.strip(), event_date.strip())
        except:
            event = ""
        record = [fighter1.strip(), fighter2.strip(), event_name.strip(), event_date.strip(), remarks.strip(), result.strip()]
        records.append(record)
        print(record)
    return records

In [117]:
df = pd.DataFrame(records, columns=["fighter1", "fighter2", "event_name", "event_date", "details", "result"])
df

,fighter1,fighter2,event_name,event_date,remarks,result
0,Dylan Andrews,Papy Abedi,UFC,"Aug 28, 2013",Condit vs. Kampmann 2 Condit vs. Kampmann 2:Li...,Loss
1,Papy Abedi,Besam Yousef,UFC Sweden 2013 Live from Stockholm,"Apr 6, 2013",Mousasi vs. Latifi,Win
2,Papy Abedi,James Head,UFC® Live,"Apr 14, 2012",Gustafsson vs. Silva,Loss
3,Thiago Alves,Papy Abedi,UFC® 138,"Nov 5, 2011",LEBEN vs. MUNOZ,Loss


In [35]:
# driver.find_element_by_xpath('//*[@id="toggleResults"]/div/a').click()
# sleep(randint(1, 2))
base_url = 'http://www.ufc.ca'

links = fighters['link'].values.tolist()
links

['/fighter/shamil-abdurakhimov',
 '/fighter/daichi-Abe',
 '/fighter/juan-Adams',
 '/fighter/israel-Adesanya',
 '/fighter/jessica-aguilar',
 '/fighter/kevin-Aguilar',
 '/fighter/omari-Akhmedov',
 '/fighter/junior-Albini',
 '/fighter/alexandra-Albu',
 '/fighter/iuri-Alcantara',
 '/fighter/hector-Aldana',
 '/fighter/irene-aldana',
 '/fighter/jose-Aldo',
 '/fighter/jj-aldrich',
 '/fighter/abdul-Razak-Alhassan',
 '/fighter/sultan-aliev',
 '/fighter/arnold-allen',
 '/fighter/thomas-almeida',
 '/fighter/eddie-alvarez',
 '/fighter/jamie-Alvarez',
 '/fighter/thiago-Alves',
 '/fighter/warlley-alves',
 '/fighter/sam-Alvey',
 '/fighter/makwan-amirkhani',
 '/fighter/eryk-Anders',
 '/fighter/corey-anderson',
 '/fighter/megan-anderson',
 '/fighter/jessica-Andrade',
 '/fighter/magomed-Ankalaev',
 '/fighter/nina-ansaroff',
 '/fighter/gadzhimurad-Antigulov',
 '/fighter/adam-Antolin',
 '/fighter/zu-Anyanwu',
 '/fighter/shinsho-anzai',
 '/fighter/josh-Appelt',
 '/fighter/felipe-Arantes',
 '/fighter/julio-

In [40]:
%%time
fighters_history = []
for link in links:
    url = base_url + link
    print(url)
    # Now just tell it wherever you want it to go
    page = requests.get(url)
    sleep(5)
    page_soup = soup(page.content, "html.parser")

    fighter_history = page_soup.find_all("tr", {"class":"fight"})
    history = get_history(fighter_history)
    print(history)    
    fighters_history.append(history)

print(fighters_history)

http://www.ufc.ca/fighter/shamil-abdurakhimov
['Andrei Arlovski', 'Shamil Abdurakhimov', 'UFC Fight Night Hunt vs Oleinik', 'Sep 15, 2018', 'Live on Fight Network', 'Win']
['Chase Sherman', 'Shamil Abdurakhimov', 'UFC Fight Night Bisping vs Gastelum', 'Nov 25, 2017', 'Live on Fight Network', 'Win']
['Derrick Lewis', 'Shamil Abdurakhimov', 'UFC Fight Night Lewis vs Abdurakhimov', 'Dec 9, 2016', 'Live on Fight Network and UFC FIGHT PASS', 'Loss']
['Shamil Abdurakhimov', 'Walt Harris', 'UFC Fight Night Lineker vs Dodson', 'Oct 1, 2016', 'Live on TSN 2', 'Win']
['Anthony Hamilton', 'Shamil Abdurakhimov', 'UFC Fight Night Cowboy vs. Cowboy', 'Feb 21, 2016', 'Live on TSN 2', 'Win']
['Shamil Abdurakhimov', 'Timothy Johnson', 'UFC Fight Night Mendes vs. Lamas', 'Apr 4, 2015', 'TSN 2', 'Loss']
[['Andrei Arlovski', 'Shamil Abdurakhimov', 'UFC Fight Night Hunt vs Oleinik', 'Sep 15, 2018', 'Live on Fight Network', 'Win'], ['Chase Sherman', 'Shamil Abdurakhimov', 'UFC Fight Night Bisping vs Gastelu

['Laureano Staropoli', 'Hector Aldana', 'UFC Fight Night', 'Nov 17, 2018', 'Magny vs. Ponzinibbio', 'UPCOMING']
['Song Kenan', 'Hector Aldana', 'UFC Fight Night Cowboy vs Edwards', 'Jun 23, 2018', 'Live on Fight Network', 'Loss']
[['Laureano Staropoli', 'Hector Aldana', 'UFC Fight Night', 'Nov 17, 2018', 'Magny vs. Ponzinibbio', 'UPCOMING'], ['Song Kenan', 'Hector Aldana', 'UFC Fight Night Cowboy vs Edwards', 'Jun 23, 2018', 'Live on Fight Network', 'Loss']]
http://www.ufc.ca/fighter/irene-aldana
['Irene Aldana', 'Lucie Pudilova', 'UFC 228 Woodley vs. Till', 'Sep 8, 2018', 'Live on Pay-Per-View', 'Win']
['Talita Bernardo', 'Irene Aldana', 'UFC Fight Night Stephens vs Choi', 'Jan 14, 2018', 'Live on TSN 5', 'Win']
['Katlyn Chookagian', 'Irene Aldana', 'UFC 210 Cormier vs. Johnson 2', 'Apr 8, 2017', 'Live on Pay-Per-View', 'Loss']
['Leslie Smith', 'Irene Aldana', 'UFC Fight Night VanZant vs Waterson', 'Dec 17, 2016', 'Live on TSN 4 & 5', 'Loss']
[['Irene Aldana', 'Lucie Pudilova', 'UFC 2

['Rick Glenn', 'Arnold Allen', 'UFC - TUF 28 Finale', 'Nov 30, 2018', '', 'UPCOMING']
['Arnold Allen', 'Mads Burnell', 'UFC Fight Night Thompson vs Till', 'May 27, 2018', 'Live on TSN 5', 'Win']
['Arnold Allen', 'Makwan Amirkhani', 'UFC Fight Night Manuwa vs Anderson', 'Mar 18, 2017', 'Live on Fight Network', 'Win']
['Arnold Allen', 'Yaotzin Meza', 'UFC Fight Night Silva vs. Bisping', 'Feb 27, 2016', 'Live on Fight Network', 'Win']
['Alan Omer', 'Arnold Allen', 'UFC Fight Night Jedrzejczyk vs. Penne', 'Jun 20, 2015', 'Fight Network', 'Win']
[['Rick Glenn', 'Arnold Allen', 'UFC - TUF 28 Finale', 'Nov 30, 2018', '', 'UPCOMING'], ['Arnold Allen', 'Mads Burnell', 'UFC Fight Night Thompson vs Till', 'May 27, 2018', 'Live on TSN 5', 'Win'], ['Arnold Allen', 'Makwan Amirkhani', 'UFC Fight Night Manuwa vs Anderson', 'Mar 18, 2017', 'Live on Fight Network', 'Win'], ['Arnold Allen', 'Yaotzin Meza', 'UFC Fight Night Silva vs. Bisping', 'Feb 27, 2016', 'Live on Fight Network', 'Win'], ['Alan Omer'

['James Krause', 'Warlley Alves', 'UFC Fight Night Gaethje vs Vick', 'Aug 25, 2018', 'Live on TSN 5', 'Loss']
['Warlley Alves', 'Sultan Aliev', 'UFC 224 Nunes vs Pennington', 'May 12, 2018', 'Live on Pay-Per-View', 'Win']
['Salim Touahri', 'Warlley Alves', 'UFC Fight Night Cerrone vs Till', 'Oct 21, 2017', 'Live on Fight Network', 'Win']
['Warlley Alves', 'Kamaru Usman', 'UFC Fight Night Bader vs Nogueira', 'Nov 19, 2016', 'Live on TSN 5', 'Loss']
['Warlley Alves', 'Bryan Barberena', 'UFC 198 Werdum vs. Miocic', 'May 14, 2016', 'Live on Pay-Per-View', 'Loss']
['Warlley Alves', 'Colby Covington', 'UFC 194 Aldo vs McGregor', 'Dec 12, 2015', 'Live on Pay-Per-View', 'Win']
['Warlley Alves', 'Nordine Taleb', 'UFC 190 Rousey vs. Correia', 'Aug 1, 2015', 'Live on Pay-Per-View', 'Win']
['Warlley Alves', 'Alan Jouban', 'UFC Fight Night Shogun vs. Saint Preux', 'Nov 8, 2014', 'UFC FIGHT PASS', 'Win']
['Marcio  Alexandre', 'Warlley Alves', 'UFC Fight Night Miocic vs. Maldonado', 'May 31, 2014', '

['Ilir Latifi', 'Corey Anderson', 'UFC 232', 'Dec 29, 2018', 'Jones vs. Gustafsson 2', 'UPCOMING']
['Glover Teixeira', 'Corey Anderson', 'UFC Fight Night Shogun vs Smith', 'Jul 22, 2018', 'Live on TSN 2', 'Win']
['Corey Anderson', 'Patrick Cummins', 'UFC Fight Night Barboza vs Lee', 'Apr 21, 2018', 'Live on TSN 5', 'Win']
['Ovince Saint Preux', 'Corey Anderson', 'UFC 217 Bisping vs St-Pierre', 'Nov 4, 2017', 'Live on Pay-Per-View', 'Loss']
['Jimi Manuwa', 'Corey Anderson', 'UFC Fight Night Manuwa vs Anderson', 'Mar 18, 2017', 'Live on Fight Network', 'Loss']
['Corey Anderson', "Sean O'Connell", 'UFC Fight Night Lewis vs Abdurakhimov', 'Dec 9, 2016', 'Live on Fight Network and UFC FIGHT PASS', 'Win']
['Mauricio Rua', 'Corey Anderson', 'UFC 198 Werdum vs. Miocic', 'May 14, 2016', 'Live on Pay-Per-View', 'Loss']
['Corey Anderson', 'Tom Lawlor', 'UFC 196 McGregor vs. Diaz', 'Mar 5, 2016', 'Live on Pay-Per-View', 'Win']
['Fabio Maldonado', 'Corey Anderson', 'UFC Fight Night Belfort vs Hende

...

['Bobby Voelker', 'Lance Benoist', 'UFC Fight Night', 'Jun 7, 2014', 'Henderson vs Khabilov', 'Loss']
['William Macario', 'Bobby Voelker', 'UFC 168 Live on Pay-Per-View', 'Dec 28, 2013', 'Weidman vs. Silva 2', 'Loss']
['Robbie Lawler', 'Bobby Voelker', 'UFC on FOX 8 Live from Seattle', 'Jul 27, 2013', 'Johnson vs. Moraga', 'Loss']
['Patrick Cote', 'Bobby Voelker', 'UFC 158 Live on Pay-Per View', 'Mar 16, 2013', 'St-Pierre vs. Diaz', 'Loss']
['Roger Bowling', 'Bobby Voelker', 'Strikeforce Challengers 17', 'Jul 22, 2011', '', 'Win']
['Roger Bowling', 'Bobby Voelker', 'Strikeforce Challengers 11', 'Oct 22, 2010', '', 'Win']
['Cory Devela', 'Bobby Voelker', 'Strikeforce Challengers 9', 'Jul 23, 2010', '', 'Win']
['Bobby Voelker', 'Roger Bowling', 'Strikeforce Challengers 8', 'May 21, 2010', '', 'Loss']
['Bobby Voelker', 'Erik Apple', 'Strikeforce Challengers 5', 'Nov 20, 2009', '', 'Win']
[['Bobby Voelker', 'Lance Benoist', 'UFC Fight Night', 'Jun 7, 2014', 'Henderson vs Khabilov', 'Loss']

['Rich Walsh', 'Jonathan Meunier', 'UFC Fight Night Whittaker vs Brunson', 'Nov 26, 2016', 'Live on TSN 5', 'Loss']
['Rich Walsh', 'Viscardi Andrade', 'UFC Fight Night Hunt vs. Mir', 'Mar 19, 2016', 'Live on TSN 2', 'Loss']
['Rich Walsh', 'Steven Kennedy', 'UFC 193 Rousey vs Holm', 'Nov 14, 2015', 'Live on Pay-Per-View', 'Win']
['Alan Jouban', 'Rich Walsh', 'UFC 184 Rousey vs. Zingano', 'Feb 28, 2015', 'Live on Pay-Per-View', 'Loss']
['Kiichi Kunimoto', 'Rich Walsh', 'UFC Fight Night Hunt vs. Nelson', 'Sep 20, 2014', 'UFC FIGHT PASS', 'Loss']
['Rich Walsh', 'Chris Indich', 'TUF Nations Finale Bisping vs. Kennedy', 'Apr 16, 2014', 'Sportsnet 360', 'Win']
[['Rich Walsh', 'Jonathan Meunier', 'UFC Fight Night Whittaker vs Brunson', 'Nov 26, 2016', 'Live on TSN 5', 'Loss'], ['Rich Walsh', 'Viscardi Andrade', 'UFC Fight Night Hunt vs. Mir', 'Mar 19, 2016', 'Live on TSN 2', 'Loss'], ['Rich Walsh', 'Steven Kennedy', 'UFC 193 Rousey vs Holm', 'Nov 14, 2015', 'Live on Pay-Per-View', 'Win'], ['Al

['Cody McKenzie', 'Aaron Wilkinson', 'The Ultimate Fighter® 12', 'Dec 4, 2010', 'Team GSP vs Team Koscheck Finale', 'Loss']
[['Cody McKenzie', 'Aaron Wilkinson', 'The Ultimate Fighter® 12', 'Dec 4, 2010', 'Team GSP vs Team Koscheck Finale', 'Loss']]
http://www.ufc.ca/fighter/mike-wilkinson
['Mike Wilkinson', 'Makwan Amirkhani', 'UFC Fight Night Silva vs. Bisping', 'Feb 27, 2016', 'Live on Fight Network', 'Loss']
['Niklas Backstrom', 'Mike Wilkinson', 'UFC Fight Night Nelson vs. Story', 'Oct 4, 2014', 'UFC FIGHT PASS', 'Win']
['Rony Jason', 'Mike Wilkinson', 'UFC on FUEL TV 10 Live from Brazil', 'Jun 8, 2013', 'Nogueira vs. Werdum', 'Loss']
['Brendan Loughnane', 'Mike Wilkinson', 'UFC on FX', 'Dec 14, 2012', 'Sotiropoulos vs. Pearson', 'Win']
[['Mike Wilkinson', 'Makwan Amirkhani', 'UFC Fight Night Silva vs. Bisping', 'Feb 27, 2016', 'Live on Fight Network', 'Loss'], ['Niklas Backstrom', 'Mike Wilkinson', 'UFC Fight Night Nelson vs. Story', 'Oct 4, 2014', 'UFC FIGHT PASS', 'Win'], ['Ron

[]
http://www.ufc.ca/fighter/jamie-Yager
['Jamie Yager', 'Rich Attonito', 'The Ultimate Fighter® FINALE', 'Jun 19, 2010', 'Team Liddell vs Team Ortiz', 'Loss']
[['Jamie Yager', 'Rich Attonito', 'The Ultimate Fighter® FINALE', 'Jun 19, 2010', 'Team Liddell vs Team Ortiz', 'Loss']]
http://www.ufc.ca/fighter/eddie-Yagin
['Mark Hominick', 'Eddie Yagin', 'UFC® 145', 'Apr 21, 2012', 'Jones vs. Evans', 'Win']
['Junior Assuncao', 'Eddie Yagin', 'UFC® 135', 'Sep 24, 2011', 'JONES vs. RAMPAGE', 'Loss']
['Vitor Ribeiro', 'Eddie Yagin', 'WFA 3', 'Nov 23, 2002', '', 'Loss']
[['Mark Hominick', 'Eddie Yagin', 'UFC® 145', 'Apr 21, 2012', 'Jones vs. Evans', 'Win'], ['Junior Assuncao', 'Eddie Yagin', 'UFC® 135', 'Sep 24, 2011', 'JONES vs. RAMPAGE', 'Loss'], ['Vitor Ribeiro', 'Eddie Yagin', 'WFA 3', 'Nov 23, 2002', '', 'Loss']]
http://www.ufc.ca/fighter/keichiro-Yamamiya
['Eugene Jackson', 'Keichiro Yamamiya', 'UFC® 23', 'Nov 19, 1999', 'Ultimate Japan 2', 'Loss']
[['Eugene Jackson', 'Keichiro Yamamiya',

['Shinsho Anzai', 'Roger Zapata', 'UFC Fight Night Nelson vs. Barnett', 'Sep 26, 2015', 'TSN 5', 'Loss']
[['Shinsho Anzai', 'Roger Zapata', 'UFC Fight Night Nelson vs. Barnett', 'Sep 26, 2015', 'TSN 5', 'Loss']]
http://www.ufc.ca/fighter/joao-Zeferino
['Joao Zeferino', 'Elias Silverio', 'UFC', 'Sep 4, 2013', 'Teixeira vs. Bader Teixeira vs. Bader:Sept. 4 on FOX Sports 1', 'Loss']
['Rafael Natal', 'Joao Zeferino', 'UFC on FX 8 Live from Brazil', 'May 18, 2013', 'Belfort vs. Rockhold', 'Loss']
[['Joao Zeferino', 'Elias Silverio', 'UFC', 'Sep 4, 2013', 'Teixeira vs. Bader Teixeira vs. Bader:Sept. 4 on FOX Sports 1', 'Loss'], ['Rafael Natal', 'Joao Zeferino', 'UFC on FX 8 Live from Brazil', 'May 18, 2013', 'Belfort vs. Rockhold', 'Loss']]
http://www.ufc.ca/fighter/yao-Zhikui
['Yao Zhikui', 'Jenel Lausa', 'UFC Fight Night Whittaker vs Brunson', 'Nov 26, 2016', 'Live on TSN 5', 'Loss']
['Yao Zhikui', 'Fredy Serrano', 'UFC Fight Night Henderson vs Masvidal', 'Nov 28, 2015', 'Fight Network', '

```
# named groups
(?<event>(?<name>.*):?(\s*)(?<date>(?<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(?<day>\d{1,2}),\s*(?<year>\d{4})))

# unnamed groups
((.*):?(\s*)((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s(\d{1,2}),\s*(\d{4})))
```

In [41]:
len(fighters_history)

1484

In [123]:
fighters_history[4]

[['Kevin Casey',
  'Ildemar Alcantara',
  'UFC Fight Night Mir vs. Duffee',
  'Jul 15, 2015',
  'TSN 4/5',
  'Loss'],
 ['Richardson Moreira',
  'Ildemar Alcantara',
  'UFC 183 Silva vs. Diaz',
  'Jan 31, 2015',
  'Live on Pay-Per-View',
  'Win'],
 ['Kenny Robertson',
  'Ildemar Alcantara',
  'UFC 175 Weidman vs. Machida',
  'Jul 5, 2014',
  'Live on Pay-Per-View',
  'Loss'],
 ['Ildemar Alcantara',
  'Albert Tumenov',
  'UFC Fight Night Live on Sportsnet 360',
  'Feb 15, 2014',
  'Machida vs. Mousasi',
  'Win'],
 ['Ildemar Alcantara',
  'Igor Araujo',
  'UFC Fight Night Live on Sportsnet',
  'Oct 9, 2013',
  'Maia vs. Shields',
  'Loss'],
 ['Ildemar Alcantara',
  'Leandro Silva',
  'UFC on FUEL TV 10 Live from Brazil',
  'Jun 8, 2013',
  'Nogueira vs. Werdum',
  'Win'],
 ['Wagner Prado',
  'Ildemar Alcantara',
  'UFC ON SPORTSNET',
  'Jan 19, 2013',
  'Belfort vs. Bisping',
  'Win']]

In [135]:
fighters.iloc[0]['name']

'Papy  Abedi'

In [136]:
fighter_df = pd.DataFrame(fighters_history[0], columns=["fighter1", "fighter2", "event_name", "event_date", "details", "result"])
fighter_df['fighter_name'] = fighters.iloc[0]['name']

fighter_df

,fighter1,fighter2,event_name,event_date,remarks,result,fighter_name
0,Dylan Andrews,Papy Abedi,UFC,"Aug 28, 2013",Condit vs. Kampmann 2 Condit vs. Kampmann 2:Li...,Loss,Papy Abedi
1,Papy Abedi,Besam Yousef,UFC Sweden 2013 Live from Stockholm,"Apr 6, 2013",Mousasi vs. Latifi,Win,Papy Abedi
2,Papy Abedi,James Head,UFC® Live,"Apr 14, 2012",Gustafsson vs. Silva,Loss,Papy Abedi
3,Thiago Alves,Papy Abedi,UFC® 138,"Nov 5, 2011",LEBEN vs. MUNOZ,Loss,Papy Abedi


In [42]:
frames = []
for i in range(0, len(fighters_history)):
    fighter_df = pd.DataFrame(fighters_history[i], columns=["fighter1", "fighter2", "event_name", "event_date", "details", "result"])
    fighter_df['fighter_name'] = fighters.iloc[i]['name']
    frames.append(fighter_df)
    
history_df = pd.concat(frames)

In [49]:
history_df['event_date'] = pd.to_datetime(history_df.event_date, infer_datetime_format=True)
history_df.sort_values(by=event_date)

,fighter1,fighter2,event_name,event_date,details,result,fighter_name
0,Andrei Arlovski,Shamil Abdurakhimov,UFC Fight Night Hunt vs Oleinik,2018-09-15,Live on Fight Network,Win,Shamil Abdurakhimov
1,Chase Sherman,Shamil Abdurakhimov,UFC Fight Night Bisping vs Gastelum,2017-11-25,Live on Fight Network,Win,Shamil Abdurakhimov
2,Derrick Lewis,Shamil Abdurakhimov,UFC Fight Night Lewis vs Abdurakhimov,2016-12-09,Live on Fight Network and UFC FIGHT PASS,Loss,Shamil Abdurakhimov
3,Shamil Abdurakhimov,Walt Harris,UFC Fight Night Lineker vs Dodson,2016-10-01,Live on TSN 2,Win,Shamil Abdurakhimov
4,Anthony Hamilton,Shamil Abdurakhimov,UFC Fight Night Cowboy vs. Cowboy,2016-02-21,Live on TSN 2,Win,Shamil Abdurakhimov
5,Shamil Abdurakhimov,Timothy Johnson,UFC Fight Night Mendes vs. Lamas,2015-04-04,TSN 2,Loss,Shamil Abdurakhimov
0,Li Jingliang,Daichi Abe,UFC Fight Night Cowboy vs Edwards,2018-06-23,Live on Fight Network,Loss,Daichi Abe
1,Luke Jumeau,Daichi Abe,UFC 221 Romero vs Rockhold,2018-02-10,Live on Pay-Per-View,Loss,Daichi Abe
2,Daichi Abe,Hyun Gyu Lim,UFC Fight Night Shogun vs Saint Preux 2,2017-09-22,Live on TSN 2 and RDS 2,Win,Daichi Abe
0,Derek Brunson,Israel Adesanya,UFC 230 Cormier vs Lewis,2018-11-03,Live on Pay-Per-View,Win,Israel Adesanya


In [53]:
history_df.sort_values(by=['event_date'], inplace=True)
history_df

,fighter1,fighter2,event_name,event_date,details,result,fighter_name
15,Royce Gracie,Ken Shamrock,UFC® 1,1993-11-12,The Beginning,Win,Royce Gracie
0,Gerard Gordeau,Teila Tuli,UFC® 1,1993-11-12,The Beginning,Loss,Teila Tuli
14,Royce Gracie,Art Jimmerson,UFC® 1,1993-11-12,The Beginning,Win,Royce Gracie
16,Royce Gracie,Gerard Gordeau,UFC® 1,1993-11-12,The Beginning,,Royce Gracie
13,Royce Gracie,Patrick Smith,UFC® 2,1994-03-11,No Way Out,,Royce Gracie
0,Patrick Smith,Ray Wizard,UFC® 2,1994-03-11,No Way Out,Loss,Ray Wizard
12,Royce Gracie,Remco Pardoel,UFC® 2,1994-03-11,No Way Out,Win,Royce Gracie
11,Royce Gracie,Jason DeLucia,UFC® 2,1994-03-11,No Way Out,Win,Royce Gracie
10,Royce Gracie,Minoki Ichihara,UFC® 2,1994-03-11,No Way Out,Win,Royce Gracie
9,Royce Gracie,Kimo Leopoldo,UFC® 3,1994-09-09,The American Dream,Win,Royce Gracie


In [147]:
sn = datetime.date.today().strftime("%B %d, %Y")
sn

'November 07, 2018'

In [52]:
history_df.to_excel("fighters_records-sorted.xlsx")


In [2]:
history_df = pd.read_excel('fighters_records.xlsx')

In [3]:
from ipysankeywidget import SankeyWidget


In [17]:
history_graph1 = history_df[['fighter1', 'fighter2', 'event_name', 'event_date']].copy()
history_graph1.columns = ['source', 'target', 'type', 'date']
history_graph1

,source,target,type
0,Fabricio Werdum,Aleksander Emelianenko,2H2H - Pride & Honor
1,Rich Franklin,Ralph Dillon,AFC 1
2,Mark Hominick,Trenell Young,Affliction - Banned
3,Andrei Arlovski,Ben Rothwell,Affliction - Banned
4,Josh Barnett,Pedro Rizzo,Affliction - Banned
5,Vitor Belfort,Terry Martin,Affliction - Banned
6,Antonio Rogerio Nogueira,Edwin DeWees,Affliction - Banned
7,Mike Pyle,JJ Ambrose,Affliction - Banned
8,Jay Hieron,Jason High,Affliction - Day of Reckoning
9,Jay Hieron,Jason High,Affliction - Day of Reckoning


In [156]:
history_graph2 = history_df[['fighter1', 'fighter2', 'event_name']].copy()
history_graph2.columns = ['source', 'target', 'type']
history_graph2

,source,target
0,UFC,Papy Abedi
1,UFC Sweden 2013 Live from Stockholm,Besam Yousef
2,UFC® Live,James Head
3,UFC® 138,Papy Abedi
0,UFC Fight Night Boetsch vs. Henderson,Jake Collier
1,UFC Fight Night Miocic vs. Maldonado,Wagner Silva
0,UFC Fight Night Henderson vs Masvidal,Alberto Mina
1,UFC Fight Night Hunt vs. Nelson,Amir Sadollah
2,UFC® 144,Jake Shields
3,UFC® 133,Yoshihiro Akiyama


In [158]:
history_graph = pd.concat([history_graph1, history_graph2])

history_graph

,source,target
0,Dylan Andrews,UFC
1,Papy Abedi,UFC Sweden 2013 Live from Stockholm
2,Papy Abedi,UFC® Live
3,Thiago Alves,UFC® 138
0,Ricardo Abreu,UFC Fight Night Boetsch vs. Henderson
1,Ricardo Abreu,UFC Fight Night Miocic vs. Maldonado
0,Yoshihiro Akiyama,UFC Fight Night Henderson vs Masvidal
1,Yoshihiro Akiyama,UFC Fight Night Hunt vs. Nelson
2,Yoshihiro Akiyama,UFC® 144
3,Vitor Belfort,UFC® 133


In [159]:
from ipysankeywidget import SankeyWidget


SankeyWidget(links=[{'target': 'UFC', 'source': 'Dylan Andrews'}, {'target': 'UFC Sweden 2013 Live from Stockh…

In [18]:
history_graph1['value'] = 1
history_graph1

,source,target,type,value
0,Fabricio Werdum,Aleksander Emelianenko,2H2H - Pride & Honor,1
1,Rich Franklin,Ralph Dillon,AFC 1,1
2,Mark Hominick,Trenell Young,Affliction - Banned,1
3,Andrei Arlovski,Ben Rothwell,Affliction - Banned,1
4,Josh Barnett,Pedro Rizzo,Affliction - Banned,1
5,Vitor Belfort,Terry Martin,Affliction - Banned,1
6,Antonio Rogerio Nogueira,Edwin DeWees,Affliction - Banned,1
7,Mike Pyle,JJ Ambrose,Affliction - Banned,1
8,Jay Hieron,Jason High,Affliction - Day of Reckoning,1
9,Jay Hieron,Jason High,Affliction - Day of Reckoning,1


In [163]:
SankeyWidget(links=history_graph1.to_dict('records'))

SankeyWidget(links=[{'type': 'UFC', 'target': 'Papy  Abedi', 'source': 'Dylan Andrews', 'value': 1}, {'type': …

In [ ]:
history_graph1.drop_duplicates(subset=['source', 'target', 'type', 'value'], keep=False, inplace = True)
history_graph1.sort_values(by=['type'], inplace=True)

In [30]:
history_graph1

,source,target,type,value
0,Fabricio Werdum,Aleksander Emelianenko,2H2H - Pride & Honor,1
1,Rich Franklin,Ralph Dillon,AFC 1,1
2,Mark Hominick,Trenell Young,Affliction - Banned,1
3,Andrei Arlovski,Ben Rothwell,Affliction - Banned,1
4,Josh Barnett,Pedro Rizzo,Affliction - Banned,1
5,Vitor Belfort,Terry Martin,Affliction - Banned,1
6,Antonio Rogerio Nogueira,Edwin DeWees,Affliction - Banned,1
7,Mike Pyle,JJ Ambrose,Affliction - Banned,1
12,Fedor Emelianenko,Andrei Arlovski,Affliction - Day of Reckoning,1
14,Vitor Belfort,Matt Lindland,Affliction - Day of Reckoning,1


In [ ]:
SankeyWidget(links=history_graph1.to_dict('records'))